<a href="https://colab.research.google.com/github/TrinStudent2025/Stable-Matching-Game/blob/main/Stable_Matching_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import random

def generate_bipartite_graph(n_left=4, n_right=4, p=0.6):
    """
    Generate a random bipartite graph.
    Left nodes are 1..n_left and right nodes are n_left+1..n_left+n_right.
    Each possible edge between a left and a right node appears with probability p.
    """
    B = nx.Graph()
    left_nodes = list(range(1, n_left+1))
    right_nodes = list(range(n_left+1, n_left+n_right+1))
    B.add_nodes_from(left_nodes, bipartite=0)
    B.add_nodes_from(right_nodes, bipartite=1)
    for u in left_nodes:
        for v in right_nodes:
            if random.random() < p:
                B.add_edge(u, v)
    return B

def get_valid_moves(graph, current_matching):
    """
    A valid move is an edge whose endpoints are not yet used in the current matching.
    """
    used_vertices = set()
    for edge in current_matching:
        used_vertices.update(edge)
    valid_moves = []
    for edge in graph.edges():
        u, v = edge
        if u not in used_vertices and v not in used_vertices:
            valid_moves.append(edge)
    return valid_moves

def is_maximal_matching(graph, matching):
    """
    Check if the current matching is maximal, i.e. no additional edge can be added.
    """
    used = set()
    for e in matching:
        used.update(e)
    for edge in graph.edges():
        u, v = edge
        if u not in used and v not in used:
            return False
    return True

def compute_maximum_matching(graph):
    """
    Compute a maximum matching using networkx (maximum cardinality).
    """
    max_match = nx.algorithms.matching.max_weight_matching(graph, maxcardinality=True)
    return max_match

def compute_minimal_maximal_matchings(graph):
    """
    Compute all maximal matchings (via backtracking) and return those with the minimum number of edges.
    For small graphs this is acceptable.
    """
    all_maximal = []
    edges = list(graph.edges())
    n = len(edges)

    def backtrack(i, current_matching, used_vertices):
        if i == n:
            # Check if current_matching is maximal:
            for e in edges:
                if e not in current_matching:
                    u, v = e
                    if u not in used_vertices and v not in used_vertices:
                        return
            all_maximal.append(set(current_matching))
            return

        # Option 1: Skip current edge
        backtrack(i+1, current_matching, used_vertices)

        # Option 2: Use current edge if possible:
        e = edges[i]
        if e[0] not in used_vertices and e[1] not in used_vertices:
            new_used = used_vertices | {e[0], e[1]}
            current_matching.append(e)
            backtrack(i+1, current_matching, new_used)
            current_matching.pop()

    backtrack(0, [], set())
    if not all_maximal:
        return None
    min_size = min(len(m) for m in all_maximal)
    min_maximals = [m for m in all_maximal if len(m) == min_size]
    return min_maximals

def display_graph(graph, current_matching):
    """
    Display the graph in text form.
    Matched edges are marked.
    """
    print("\nNodes:", sorted(graph.nodes()))
    print("Edges:")
    for edge in graph.edges():
        # Because the graph is undirected, check both orderings
        if edge in current_matching or (edge[1], edge[0]) in current_matching:
            marker = " [MATCHED]"
        else:
            marker = ""
        print(f"  {edge}{marker}")
    print("\nCurrent Matching:", current_matching)
    used = set()
    for e in current_matching:
        used.update(e)
    print("Used vertices:", used)
    print("-" * 40)

def main():
    print("Welcome to the Bipartite Matching Game!\n")
    print("Game Rules:")
    print("  - You are given a bipartite graph. Left nodes are numbered 1..4 and right nodes 5..8.")
    print("  - A matching is a set of edges with no shared vertices.")
    print("  - On each turn, select an available edge (one whose endpoints are both free) by its index.")
    print("  - The game ends when no valid moves remain (a maximal matching is reached).\n")

    while True:
        # Generate a random bipartite graph.
        graph = generate_bipartite_graph(n_left=4, n_right=4, p=0.6)
        # Ensure the graph has at least one edge.
        if len(graph.edges()) == 0:
            continue

        current_matching = set()

        while True:
            display_graph(graph, current_matching)
            valid_moves = get_valid_moves(graph, current_matching)
            if not valid_moves:
                print("No more valid moves available. Maximal matching reached.\n")
                break

            print("Available Moves:")
            for i, edge in enumerate(valid_moves):
                print(f"  {i}: {edge}")

            move = input("Enter the number of the edge you want to select (or 'q' to quit this game): ")
            if move.lower() == 'q':
                print("Game aborted.")
                return

            try:
                move_index = int(move)
                if move_index < 0 or move_index >= len(valid_moves):
                    print("Invalid move index. Try again.\n")
                    continue
            except ValueError:
                print("Please enter a valid number.\n")
                continue

            chosen_edge = valid_moves[move_index]
            current_matching.add(chosen_edge)
            print(f"\nEdge {chosen_edge} added to matching.\n")

        # Game over: display final matching and comparisons.
        print("Final Matching:", current_matching)
        max_match = compute_maximum_matching(graph)
        print("Maximum Matching (computed using networkx):", max_match)

        if len(current_matching) * 2 == len(graph.nodes()):
            print("Your matching is a Perfect Matching!")
        else:
            print("Your matching is NOT a Perfect Matching.")

        min_maximals = compute_minimal_maximal_matchings(graph)
        if min_maximals is not None and len(min_maximals) > 0:
            print("One example of a Minimum Maximal Matching:", min_maximals[0])
        else:
            print("Could not compute a Minimum Maximal Matching for this graph.")

        print("\nKey Definitions:")
        print("  Matching: A set of edges with no two sharing a vertex.")
        print("  Maximal Matching: A matching that cannot be extended by adding any other edge.")
        print("  Maximum Matching: A matching with the largest possible number of edges.")
        print("  Perfect Matching: A matching that covers every vertex in the graph.")
        print("  Minimum Maximal Matching (MMM): Among all maximal matchings, one with the fewest edges.\n")

        replay = input("Do you want to play again? (y/n): ")
        if replay.lower() != 'y':
            print("Thanks for playing!")
            break
        print("\n" + "=" * 50 + "\n")

if __name__ == '__main__':
    main()


Welcome to the Bipartite Matching Game!

Game Rules:
  - You are given a bipartite graph. Left nodes are numbered 1..4 and right nodes 5..8.
  - A matching is a set of edges with no shared vertices.
  - On each turn, select an available edge (one whose endpoints are both free) by its index.
  - The game ends when no valid moves remain (a maximal matching is reached).


Nodes: [1, 2, 3, 4, 5, 6, 7, 8]
Edges:
  (1, 6)
  (1, 8)
  (2, 6)
  (2, 7)
  (2, 8)
  (3, 5)
  (3, 6)
  (3, 7)
  (4, 5)
  (4, 6)
  (4, 8)

Current Matching: set()
Used vertices: set()
----------------------------------------
Available Moves:
  0: (1, 6)
  1: (1, 8)
  2: (2, 6)
  3: (2, 7)
  4: (2, 8)
  5: (3, 5)
  6: (3, 6)
  7: (3, 7)
  8: (4, 5)
  9: (4, 6)
  10: (4, 8)
Enter the number of the edge you want to select (or 'q' to quit this game): 10

Edge (4, 8) added to matching.


Nodes: [1, 2, 3, 4, 5, 6, 7, 8]
Edges:
  (1, 6)
  (1, 8)
  (2, 6)
  (2, 7)
  (2, 8)
  (3, 5)
  (3, 6)
  (3, 7)
  (4, 5)
  (4, 6)
  (4, 8) [M

KeyboardInterrupt: Interrupted by user

In [22]:
import networkx as nx
import random
import matplotlib.pyplot as plt
from ipywidgets import Button, Output, VBox, HBox, Layout
from IPython.display import display

class VisualMatchingGame:
    def __init__(self, n_left=4, n_right=4, p=0.6):
        """
        Initialize game parameters and UI widgets.
          - n_left, n_right: Number of nodes on each side (bipartite graph).
          - p: Probability for an edge between a left and right node.
        """
        self.n_left = n_left
        self.n_right = n_right
        self.p = p
        self.graph = None
        self.current_matching = set()

        # Output widgets for graph display and messages.
        self.graph_out = Output()
        self.msg_out = Output()
        # Container for move buttons.
        self.moves_box = HBox([])

        # Restart button to start a new game.
        self.restart_button = Button(description="Restart", button_style='warning')
        self.restart_button.on_click(self.restart_game)

        # Main container to hold everything.
        self.container = VBox([self.graph_out, self.msg_out, self.moves_box, self.restart_button])

        # Start the game.
        self.reset_game()
        display(self.container)

    def reset_game(self):
        """Generates a new bipartite graph and resets the matching."""
        self.graph = self.generate_bipartite_graph()
        self.current_matching = set()
        self.update_ui()

    def generate_bipartite_graph(self):
        """
        Generate a random bipartite graph.
        Left nodes are numbered 1..n_left and right nodes are numbered n_left+1..n_left+n_right.
        """
        B = nx.Graph()
        left_nodes = list(range(1, self.n_left+1))
        right_nodes = list(range(self.n_left+1, self.n_left+self.n_right+1))
        B.add_nodes_from(left_nodes, bipartite=0)
        B.add_nodes_from(right_nodes, bipartite=1)
        for u in left_nodes:
            for v in right_nodes:
                if random.random() < self.p:
                    B.add_edge(u, v)
        return B

    def get_valid_moves(self):
        """
        Return a list of edges that can be added to the matching.
        An edge is valid if both its endpoints are not already matched.
        """
        used = set()
        for edge in self.current_matching:
            used.update(edge)
        valid = []
        for edge in self.graph.edges():
            if edge[0] not in used and edge[1] not in used:
                valid.append(edge)
        return valid

    def update_ui(self):
        """Refresh the graph display, available moves, and status messages."""
        self.draw_graph()
        valid_moves = self.get_valid_moves()

        self.msg_out.clear_output()
        with self.msg_out:
            print("Current Matching:", self.current_matching)

        # If valid moves exist, create buttons for each.
        if valid_moves:
            move_buttons = []
            for edge in valid_moves:
                btn = Button(
                    description=f"{edge[0]}-{edge[1]}",
                    layout=Layout(width='80px')
                )
                # Use a lambda default argument to capture the current edge.
                btn.on_click(lambda b, edge=edge: self.make_move(edge))
                move_buttons.append(btn)
            self.moves_box.children = move_buttons
        else:
            # No valid moves remain: the matching is maximal.
            self.moves_box.children = []
            self.msg_out.clear_output()
            with self.msg_out:
                print("No more valid moves available. Maximal matching reached.\n")
                print("Final Matching:", self.current_matching)
                max_match = nx.algorithms.matching.max_weight_matching(self.graph, maxcardinality=True)
                print("Maximum Matching (computed by networkx):", max_match)
                if len(self.current_matching)*2 == len(self.graph.nodes()):
                    print("Your matching is a Perfect Matching!")
                else:
                    print("Your matching is NOT a Perfect Matching.")
                min_maximals = self.compute_minimal_maximal_matchings()
                if min_maximals:
                    print("One example of a Minimum Maximal Matching:", list(min_maximals[0]))
                else:
                    print("Could not compute a Minimum Maximal Matching for this graph.")
                print("\nKey Definitions:")
                print("  Matching: A set of edges with no two sharing a vertex.")
                print("  Maximal Matching: A matching that cannot be extended by adding any other edge.")
                print("  Maximum Matching: The matching with the largest number of edges.")
                print("  Perfect Matching: A matching that covers every vertex in the graph.")
                print("  Minimum Maximal Matching (MMM): Among all maximal matchings, one with the fewest edges.")

    def draw_graph(self):
        """Display the current graph and highlight the matching edges in red."""
        self.graph_out.clear_output(wait=True)
        with self.graph_out:
            plt.figure(figsize=(6,6))
            # Use the bipartite layout based on left nodes.
            pos = nx.bipartite_layout(self.graph, nodes=range(1, self.n_left+1))
            # Draw all edges in light gray.
            nx.draw(self.graph, pos, with_labels=True, node_color='lightblue',
                    edge_color='gray', node_size=500)
            # Highlight matched edges in red.
            if self.current_matching:
                nx.draw_networkx_edges(self.graph, pos, edgelist=list(self.current_matching),
                                       edge_color='red', width=2)
            plt.show()

    def make_move(self, edge):
        """Add the selected edge to the matching and update the UI."""
        self.current_matching.add(edge)
        self.update_ui()

    def restart_game(self, b):
        """Handler for the restart button; resets the game."""
        self.reset_game()

    def compute_minimal_maximal_matchings(self):
        """
        Compute all maximal matchings (via backtracking) and return those with the minimum number of edges.
        Suitable for small graphs.
        """
        all_maximal = []
        edges = list(self.graph.edges())
        n = len(edges)

        def backtrack(i, current_matching, used):
            if i == n:
                # Check if the current matching is maximal.
                for e in edges:
                    if e not in current_matching:
                        if e[0] not in used and e[1] not in used:
                            return
                all_maximal.append(set(current_matching))
                return

            # Option 1: Skip current edge.
            backtrack(i+1, current_matching, used)

            # Option 2: Use current edge if possible.
            e = edges[i]
            if e[0] not in used and e[1] not in used:
                new_used = used | {e[0], e[1]}
                current_matching.append(e)
                backtrack(i+1, current_matching, new_used)
                current_matching.pop()

        backtrack(0, [], set())
        if not all_maximal:
            return None
        min_size = min(len(m) for m in all_maximal)
        min_maximals = [m for m in all_maximal if len(m) == min_size]
        return min_maximals

# Create an instance of the game.
game = VisualMatchingGame()
